In [35]:
import os

import torch
import torch.nn as nn
from torch.nn.functional import softmax
import torch.optim as optim
import torchvision
from torchvision import transforms, models

from PIL import Image
from glob import glob
import pandas as pd
from time import time as timer
import numpy as np
import multiprocessing

In [36]:
def predict(model,img_paths, img_shape, batch_size = 1, num_workers = 1, apply_softmax = True):
    """ get multi-class model predictions from a pytorch model for a set of images
    
    model: a pytorch model object (not path to weights)
    img_paths: a list of paths to RGB png spectrograms
    img_shape: [x,y] shape of images expected by model
    batch_size=1: parallelization parameter (see torch.utils.data.DataLoader)
    num_workers=1: parallelization parameter (see torch.utils.data.DataLoader)
    apply_softmax=True: if True, puts output of network through a softmax layer
    
    returns: df of predictions indexed by file (columns=class names? #TODO)"""
    class PredictionDataset(torch.utils.data.Dataset):
        def __init__(self, df, height=img_shape[0], width=img_shape[1]):
            self.data = df

            self.height = height
            self.width = width

            self.mean = torch.tensor([0.8013 for _ in range(3)]) 
            self.std_dev = ([0.1576 for _ in range(3)])

            self.transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(self.mean, self.std_dev)
            ])

        def __len__(self):
            return self.data.shape[0]

        def __getitem__(self, idx):
            row = self.data.iloc[idx, :]
            image = Image.open(row["filename"])
            image = image.convert("RGB")
            image = image.resize((self.width, self.height))

            return self.transform(image) #, torch.from_numpy(labels)

    #turn list of files into a df (this maintains parallelism with training format)
    file_df = pd.DataFrame(columns=['filename'],data=img_paths)
    
    #create pytorch dataset and dataloader objects
    dataset = PredictionDataset(file_df)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    
    # run prediction
    all_predictions = []
    with torch.no_grad():
        for i, inputs in enumerate(dataloader):
            predictions = model(inputs)
            if apply_softmax:
                softmax_val = softmax(predictions, 1).detach().cpu().numpy().tolist()#.astype('float64'))
                all_predictions += softmax_val
            else:
                all_predictions += predictions.detach().numpy().tolist() #.astype('float64')

    #how do we get class names? are they saved in the file?
    #the column names should be class names    
    return all_predictions #pd.DataFrame(index=img_paths,data=all_predictions)

In [43]:
# Load Model
species_sci_name = 'empidonax-virescens'
IMAGE_SHAPE = (224,224)

models_dir = '/Volumes/seagate3/single-species-models/models'
model_weights_path = os.path.join(models_dir, f'{species_sci_name}-epoch-4.model')
num_classes=2
model = torchvision.models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load(model_weights_path))
model.eval();

In [54]:
# we want to use multithreading

In [52]:
%env OMP_NUM_THREAD=12

env: OMP_NUM_THREAD=12


In [53]:
# images = pd.read_csv('/Volumes/seagate2/LOCA/4.1.0_with_background/8_events/sim_3/clip_df.csv').img_path.values
images = glob('../../OPSO/data/pnre-5s/*.png')

images = images[0:100]

#predict  
t0 = timer()
predictions = predict(model,images, IMAGE_SHAPE, apply_softmax=True, batch_size=20, num_workers=12)
print(f'prediction of {len(images)} images took {timer()-t0} seconds')

prediction of 100 images took 9.822107076644897 seconds


In [7]:
np.shape(predictions)

(2,)